In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
stemmer = PorterStemmer()

def cut_sentence(line):

    words = word_tokenize(line)
    toRet = [] 
    for word in words:
        toRet.append(word)
        if word in ["depression", "depressed"]:
            break
    return toRet

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\73183\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\73183\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
import json
import os

def parameter_tuning(rate1):
    counter = 0
    current = os.getcwd()
    neg_users = os.listdir(os.path.join(current,'new_ds', 'negative'))
    sia = SentimentIntensityAnalyzer()
    rates = [0.1*i for i in range(1,7)]
    rate_counter = [0 for i in range(1,7)]
    rate_dic = dict(zip(rates,rate_counter))
    new_pos = []


    for user in neg_users:
        text_dir = os.path.join(current,'new_ds', 'negative', user, 'timeline.txt')
        user_counter = 0
        with open(text_dir) as f:

            tweets = f.readlines()
            total = len(tweets)
            if total == 0:
                continue
            for tweet in tweets:

                j_tweet = json.loads(tweet)
                text = j_tweet['full_text']


                list = cut_sentence(text)
                sentence = " ".join(list)
                if sia.polarity_scores(sentence)['compound'] < rate1:
                    #print(sentence)
                    #print(text)
                    user_counter+= 1
        
        for rate2 in rates:
            if (user_counter/total) > rate2:
                rate_dic[rate2] += 1

    for rate, rate_count in rate_dic.items():
        print("{:15} {:15} {:15}".format(round(rate1,2), round(rate,2), rate_count))

    return counter

In [24]:
rates = [-0.1*i for i in range(1,7)]
print("{:15} {:15} {:15}".format("compound_score", "tweet_ratio", "number_of_new_positive"))
for rate1 in rates:

    score = parameter_tuning(rate1)



compound_score  tweet_ratio     number_of_new_positive
           -0.1             0.1             791
           -0.1             0.2             354
           -0.1             0.3              82
           -0.1             0.4               9
           -0.1             0.5               2
           -0.1             0.6               2
           -0.2             0.1             723
           -0.2             0.2             253
           -0.2             0.3              54
           -0.2             0.4               4
           -0.2             0.5               2
           -0.2             0.6               2
           -0.3             0.1             615
           -0.3             0.2             153
           -0.3             0.3              20
           -0.3             0.4               3
           -0.3             0.5               2
           -0.3             0.6               2
           -0.4             0.1             503
           -0.4             0.2  

take -0.6, 0.1 as threshold, gives 117 more users.

In [2]:
def dig():
    counter = 0
    current = os.getcwd()
    neg_users = os.listdir(os.path.join(current,'new_ds', 'negative'))
    sia = SentimentIntensityAnalyzer()
    new_pos = []


    for user in neg_users:
        text_dir = os.path.join(current,'new_ds', 'negative', user, 'timeline.txt')
        user_counter = 0
        with open(text_dir) as f:

            tweets = f.readlines()
            total = len(tweets)
            if total == 0:
                continue
            for tweet in tweets:

                j_tweet = json.loads(tweet)
                text = j_tweet['full_text']


                list = cut_sentence(text)
                sentence = " ".join(list)
                if sia.polarity_scores(sentence)['compound'] < (-0.6):
                    #print(sentence)
                    #print(text)
                    user_counter+= 1
        

        if (user_counter/total) > 0.1:
            new_pos.append(user)


    return new_pos

In [5]:
new_pos = dig()

In [7]:
import shutil

current = os.getcwd()

for user in new_pos:
    ori = os.path.join(current,'new_ds', 'negative', user)
    target = os.path.join(current,'new_ds', 'positive', user)
    shutil.move(ori,target)

In [2]:

import torch